In [1]:
import parse_inventory as pi
import parse_dealership
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Dealership Info Dictionary
dealerships = {
    'Bostonyan Auto Group': {
        'url': 'https://www.bostonyanautogroup.com/view-inventory',
        'pagination_url': '',
        'dealership_name': 'Bostonyan Auto Group',
        'address': '119 Worcester St',
        'zipcode': '01760',
        'city': 'Natick',
        'state': 'MA'
    },
    'Direct Auto Mecca': {
        'url': 'https://www.directautomecca.com/view-inventory.aspx',
        'pagination_url': 'https://www.directautomecca.com/inventory.aspx?_new=true&_used=true&_page=2',
        'dealership_name': 'Direct Auto Mecca',
        'address': '154 Waverly Street',
        'zipcode': '01760',
        'city': 'Natick',
        'state': 'MA'
    },
    'Fafama Auto Sales': {
        'url': 'https://www.fafama.com/used-cars.aspx',
        'pagination_url': 'https://www.fafama.com/inventory.aspx?_used=true&_page=2',
        'dealership_name': 'Fafama Auto Sales',
        'address': '5 Cape Road',
        'zipcode': '01757',
        'city': 'Milford',
        'state': 'MA'
    },
    'Newton Automotive Sales': {
        'url': 'https://www.newtonautoandsales.com/cars-for-sale',
        'pagination_url': 'https://www.newtonautoandsales.com/cars-for-sale?PageNumber=2&Sort=MakeAsc&StockNumber=&Condition=&BodyStyle=&Make=&MaxPrice=&Mileage=&SoldStatus=AllVehicles&StockNumber=',
        'dealership_name': 'Newton Automotive Sales',
        'address': '249 Centre Street',
        'zipcode': '02458',
        'city': 'Newton',
        'state': 'MA'
    },
    'Blasius Boston': {
        'url': 'https://www.blasiusboston.com/used-cars-holliston-ma',
        'pagination_url': 'https://www.blasiusboston.com/used-cars-holliston-ma?page=2',
        'dealership_name': 'Blasius Boston',
        'address': '1286 Washington Street',
        'zipcode': '01746',
        'city': 'Holliston',
        'state': 'MA'
    },
    'Avon Auto Brokers': {
        'url': 'https://avonautobrokers.com/newandusedcars?clearall=1',
        'pagination_url': 'https://avonautobrokers.com/newandusedcars?page=2',
        'dealership_name': 'Avon Auto Brokers',
        'address': '159 Memorial Drive',
        'zipcode': '02322',
        'city': 'Avon',
        'state': 'MA'
    }, 
    'Johns Auto Sales': {
        'url': 'https://johnsautosales.com/newandusedcars?clearall=1',
        'pagination_url': 'https://johnsautosales.com/newandusedcars?page=2',
        'dealership_name': "John's Auto Sales",
        'address': '181 Somerville Avenue',
        'zipcode': '02143',
        'city': 'Somerville',
        'state': 'MA'
    },
    'JM Automotive': {
        'url': 'https://www.jmautomotive.com/cars-for-sale-in-Naugatuck-CT-Hartford-New-Haven/used_cars',
        'pagination_url': 'https://www.jmautomotive.com/inventory.aspx?pg=2&sort=12&limit=50&vstatus=1&status=6',
        'dealership_name': 'J&M Automotive',
        'address': '756/820 New Haven Road',
        'zipcode': '06770',
        'city': 'Naugatuck',
        'state': 'CT'
    }
}


## Test Connection

Check that the html for the page is being pulled in correctly

In [3]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
}

URL = 'https://www.jmautomotive.com/cars-for-sale-in-Naugatuck-CT-Hartford-New-Haven/used_cars'
#URL = 'https://www.jmautomotive.com/inventory.aspx?pg=11&sort=12&limit=50&vstatus=1&status=6'
response = requests.get(URL, headers = headers)
soup = BeautifulSoup(response.text, "html.parser")


In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-us">
 <head>
  <title>
   Used cars for sale in Naugatuck, Waterbury, Hartford, New Haven, CT | J&amp;M Automotive Sls&amp;Svc LLC
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="black" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="width=device-width, user-scalable=yes, initial-scale=1.0, minimum-scale=1.0, maximum-scale=9.0" name="viewport"/>
  <meta content="index, follow" name="robots"/>
  <meta content="J&amp;M Automotive Sls&amp;Svc LLC, used cars for sale, used car dealer, Connecticut, CT, Naugatuck, Waterbury, Hartford, New Haven, New Haven County, Waterbury CT, Hartford CT, Prospect CT, Oxford CT, Seymour CT, Cheshire CT, Southbury CT, Wolcott CT, Watertown CT, Woodbury CT, Woodbridge CT, Ansonia CT, Hamden CT, Derby CT, North Haven CT and Shelton CT,  used cars, used suv, used trucks, used pickups, used minivans, car financing, service and repair center in Naugatuck, CT" name="k

## Test Extracting Columns

### Title

In [5]:
pi.parse_subsection_attr(soup, 'title','div', 'a', 'thumbnail', 'listitemlink')

['Used 2016 Ford F-150 4WD SuperCrew 145 XLT for sale in Naugatuck, CT',
 'Used 2016 Ford Super Duty F-350 SRW 4WD Crew Cab 156 Platinum for sale in Naugatuck, CT',
 'Used 2017 Mini Countryman Cooper ALL4 for sale in Naugatuck, CT',
 'Used 2015 Ford Expedition 4WD 4dr Platinum for sale in Naugatuck, CT',
 'Used 2017 Infiniti Q50 3.0t Premium AWD for sale in Naugatuck, CT',
 'Used 2015 Ford Super Duty F-250 SRW 4WD Crew Cab 156 XL for sale in Naugatuck, CT',
 'Used 2018 Volkswagen Jetta 1.4T SE Auto for sale in Naugatuck, CT',
 'Used 2015 Toyota Tundra Double Cab 5.7L V8 6-Spd AT SR for sale in Naugatuck, CT',
 'Used 2013 Audi TTS 2dr Cpe S tronic quattro 2.0T Prest for sale in Naugatuck, CT',
 'Used 2015 GMC Yukon 4WD 4dr Denali for sale in Naugatuck, CT',
 'Used 2013 Jeep Wrangler Unlimited 4WD 4dr Sport for sale in Naugatuck, CT',
 'Used 2014 Audi S4 4dr Sdn S Tronic Prestige for sale in Naugatuck, CT',
 'Used 1993 Chevrolet Corvette 2dr Convertible for sale in Naugatuck, CT',
 'Used

### Year

In [6]:

col_data = []
main = soup.findAll('div', 'thumbnail')
sub_section_attr_parse_key = 'itemprop'
sub_section_attr_parse_value = 'vehicleInteriorColor'
for el in main:
    sub_data = el.findAll()
    appendedData = False
    for sd in sub_data:
        if sub_section_attr_parse_key is not None and sub_section_attr_parse_value is not None:
            if sd.get(sub_section_attr_parse_key) == sub_section_attr_parse_value:
                col_data.append(sd.getText())
                appendedData = True
        else:
            col_data.append(sd.getText())
            appendedData = True
        
    if (appendedData == False):
        col_data.append(None)

col_data

['Medium Earth Gray  ',
 'Pecan ',
 'Satellite Grey  ',
 'Brunell W/black  ',
 'Graphite  ',
 'Steel ',
 'Titan Black  ',
 'Graphite  ',
 'Black ',
 'Cocoa/dark Atmosphe  ',
 'Black Interior  ',
 'Black/red  ',
 None,
 'Silk Beige/espresso  ',
 'Ebony  ',
 'Black  ',
 'Beige  ',
 'Black  ',
 'Dark Charcoal  ',
 'Beige  ',
 'Off-black  ',
 'Ivory  ',
 'Black  ',
 'Black  ',
 'Ebony  ',
 'Cafe Latte  ',
 'Charcoal Black  ',
 'Charcoal  ',
 'Black/light Graysto  ',
 'Jet Black/light Whe  ',
 'Gray  ',
 'Beige  ',
 'Wheat  ',
 'Redwood  ',
 'Black  ',
 'Black  ',
 'Jet Black  ',
 'Jet Black  ',
 'Jet Black  ',
 'Ash  ',
 'Black  ',
 'Jet Black/jet Black  ',
 'Pale Adobe  ',
 'Charcoal Black  ',
 'Black  ',
 'Gray  ',
 'Cardamom  ',
 'Ebony  ',
 'Charcoal Black  ',
 'Black Interior  ']

In [7]:
pi.parse_main_section_attr_text_all(soup, 'div', 'thumbnail', 'itemprop', 'vehicleModelDate')

['2016',
 '2016',
 '2017',
 '2015',
 '2017',
 '2015',
 '2018',
 '2015',
 '2013',
 '2015',
 '2013',
 '2014',
 '1993',
 '2014',
 '2004',
 '2017',
 '2009',
 '2016',
 '2014',
 '2016',
 '2018',
 '2012',
 '2018',
 '2019',
 '2020',
 '2012',
 '2014',
 '2016',
 '2016',
 '2013',
 '2016',
 '2015',
 '2018',
 '2015',
 '2014',
 '2014',
 '2015',
 '2016',
 '2019',
 '2018',
 '2020',
 '2018',
 '2013',
 '2015',
 '2018',
 '2017',
 '2015',
 '2019',
 '2018',
 '2013']

### Make

In [8]:
pi.parse_main_section_attr_text_all(soup, 'div', 'thumbnail', 'itemprop', 'manufacturer')

['Ford',
 'Ford',
 'Mini',
 'Ford',
 'Infiniti',
 'Ford',
 'Volkswagen',
 'Toyota',
 'Audi',
 'GMC',
 'Jeep',
 'Audi',
 'Chevrolet',
 'Mercedes-benz',
 'Chevrolet',
 'Nissan',
 'Honda',
 'Lexus',
 'Toyota',
 'Hyundai',
 'Volvo',
 'Honda',
 'Dodge',
 'Dodge',
 'Acura',
 'Nissan',
 'Ford',
 'Nissan',
 'Chrysler',
 'Cadillac',
 'Honda',
 'Kia',
 'Infiniti',
 'Toyota',
 'Lexus',
 'Audi',
 'GMC',
 'Chevrolet',
 'Chevrolet',
 'Toyota',
 'Mazda',
 'Chevrolet',
 'Ford',
 'Ford',
 'Toyota',
 'Honda',
 'Audi',
 'Acura',
 'Ford',
 'Fiat']

### Model

In [9]:
pi.parse_main_section_attr_text_all(soup, 'div', 'thumbnail', 'itemprop', 'model')

['F-150',
 'Super Duty F-350 SRW',
 'Countryman',
 'Expedition',
 'Q50',
 'Super Duty F-250 SRW',
 'Jetta',
 'Tundra',
 'TTS',
 'Yukon',
 'Wrangler Unlimited',
 'S4',
 'Corvette',
 'E-class',
 'Ssr',
 'Titan',
 'Ridgeline',
 'Rx 350',
 'Fj Cruiser',
 'Genesis',
 'S60',
 'Accord',
 'Challenger',
 'Challenger',
 'Tlx',
 'Maxima',
 'Fusion',
 'Maxima',
 'Town & Country',
 'Xts',
 'Pilot',
 'Optima',
 'Q50',
 '4runner',
 'Es 350',
 'S4',
 'Sierra 1500',
 'Silverado 1500',
 'Colorado',
 'Sienna',
 'Cx-5',
 'Volt',
 'F-150',
 'Taurus',
 'Rav4',
 'Cr-v',
 'Q7',
 'Tlx',
 'Taurus',
 '500']

### Trim

In [10]:
pi.parse_main_section_attr_text_all(soup, 'div', 'thumbnail', 'itemprop', 'vehicleConfiguration')

['4WD SuperCrew 145 XLT',
 '4WD Crew Cab 156 Platinum',
 'Cooper ALL4',
 '4WD 4dr Platinum',
 '3.0t Premium AWD',
 '4WD Crew Cab 156 XL',
 '1.4T SE Auto',
 'Double Cab 5.7L V8 6-Spd AT SR',
 '2dr Cpe S tronic quattro 2.0T Prest',
 '4WD 4dr Denali',
 '4WD 4dr Sport',
 '4dr Sdn S Tronic Prestige',
 '2dr Convertible',
 '2dr Cpe E 350 4MATIC',
 'Reg Cab 116.0 WB LS',
 '4x4 Crew Cab SV',
 '4WD Crew Cab RTL',
 'AWD 4dr',
 '4WD 4dr Auto',
 '4dr Sdn V6 3.8L AWD',
 'T5 AWD Dynamic',
 '4dr I4 Auto LX',
 'GT AWD',
 'SXT AWD',
 '3.5L SH-AWD w/Technology Pkg',
 '4dr Sdn V6 CVT 3.5 SV',
 '4dr Sdn SE FWD',
 '4dr Sdn 3.5 Platinum',
 '4dr Wgn Touring',
 '4dr Sdn Platinum AWD',
 'AWD 4dr EX',
 '4dr Sdn LX',
 '3.0t LUXE AWD',
 '4WD 4dr V6 Limited',
 '4dr Sdn',
 '4dr Sdn Man Premium Plus',
 '4WD Crew Cab 143.5 SLE',
 '4WD Double Cab 143.5 LTZ w/1LZ',
 '4WD Crew Cab 128.3 LT',
 'LE FWD 8-Passenger',
 'Grand Touring AWD',
 '5dr HB LT',
 '4WD SuperCrew 145 XLT',
 '4dr Sdn SEL FWD',
 'XLE AWD',
 'EX-L AWD w/N

### Mileage

In [11]:
pi.convert_to_numeric_type(
pi.parse_subsection(soup, 'div', 'li', 'thumbnail', 'list-group-item mileage', 'get_text'))

[89305,
 24752,
 77144,
 87901,
 49387,
 59080,
 37612,
 82527,
 89390,
 75993,
 67146,
 95490,
 38420,
 68064,
 64413,
 76087,
 72027,
 27063,
 63318,
 78040,
 31413,
 76673,
 34654,
 28332,
 12393,
 52336,
 76848,
 82178,
 74716,
 82281,
 59209,
 74167,
 39641,
 76652,
 78813,
 81623,
 89543,
 63285,
 44214,
 93842,
 23256,
 29971,
 99811,
 87013,
 64455,
 94402,
 84066,
 23110,
 96662,
 67502]

### Car Prices

In [12]:
pi.convert_to_numeric_type(pi.parse_subsection(soup, 'div', 'div', 'thumbnail', 'pricetag', 'get_text'))

[28995,
 52995,
 17995,
 30995,
 28995,
 42995,
 19995,
 31995,
 23995,
 39995,
 25995,
 26995,
 15995,
 25995,
 21995,
 32995,
 23995,
 36995,
 38995,
 24995,
 27995,
 15995,
 32995,
 31995,
 37995,
 17995,
 14995,
 22995,
 18995,
 22995,
 28995,
 15995,
 30995,
 35995,
 22995,
 30995,
 30995,
 36995,
 35995,
 25995,
 32995,
 23995,
 24995,
 15995,
 26995,
 24995,
 26995,
 31995,
 19995,
 9995]

### Exterior Color

In [13]:
pi.clean_text_data(
    pi.parse_subsection(soup, 'div', 'li', 'thumbnail', 'list-group-item InvExteriorcolor', 'get_text'), 'Exterior')

['Ingot Silver Metallic',
 'Shadow Black',
 'Moonwalk Gray Metallic',
 'Ruby Red Metallic Tinted Clearcoat',
 'Majestic White',
 'Oxford White',
 'White Silver Metallic',
 'Black',
 'Scuba Blue Metallic',
 'Crystal Red Tintcoat',
 'Billet Silver Metallic',
 'Brilliant Black',
 'Green',
 'Polar White',
 'Redline Red',
 'Magnetic Black',
 'Nighthawk Black Pearl',
 'Nightfall Mica',
 'Quicksand',
 'Pamplona Red',
 'Passion Red',
 'Dark Amber Metallic',
 'White Knuckle Clearcoat',
 'Octane Red Pearlcoat',
 'Platinum White Pearl',
 'Navy Blue Metallic',
 'Tuxedo Black Metallic',
 'Forged Bronze',
 'Deep Cherry Red Crystal Pearlcoat',
 'Black Raven',
 'Obsidian Blue Pearl',
 'Dark Cherry',
 'Majestic White',
 'Blizzard Pearl',
 'Obsidian',
 'Glacier White Metallic',
 'Stone Blue Metallic',
 'Black',
 'Silver Ice Metallic',
 'Midnight Black Metallic',
 'Deep Crystal Blue Mica',
 'Summit White',
 'Pale Adobe Metallic',
 'Tuxedo Black Metallic',
 'Black',
 'Gunmetal Metallic',
 'Atlantis Blue M

### Interior Color

In [14]:
pi.clean_text_data(pi.parse_subsection(soup, 'div', 'li', 'thumbnail', 'list-group-item InvInteriorcolor', 'get_text'), 'Interior')

['Medium Earth Gray',
 'Pecan',
 'Satellite Grey',
 'Brunell W/black',
 'Graphite',
 'Steel',
 'Titan Black',
 'Graphite',
 'Black',
 'Cocoa/dark Atmosphe',
 'Black Interior',
 'Black/red',
 None,
 'Silk Beige/espresso',
 'Ebony',
 'Black',
 'Beige',
 'Black',
 'Dark Charcoal',
 'Beige',
 'Off-black',
 'Ivory',
 'Black',
 'Black',
 'Ebony',
 'Cafe Latte',
 'Charcoal Black',
 'Charcoal',
 'Black/light Graysto',
 'Jet Black/light Whe',
 'Gray',
 'Beige',
 'Wheat',
 'Redwood',
 'Black',
 'Black',
 'Jet Black',
 'Jet Black',
 'Jet Black',
 'Ash',
 'Black',
 'Jet Black/jet Black',
 'Pale Adobe',
 'Charcoal Black',
 'Black',
 'Gray',
 'Cardamom',
 'Ebony',
 'Charcoal Black',
 'Black Interior']

In [15]:
pi.clean_text_data(
    pi.parse_main_section_attr_text_all(soup, 'div', 'thumbnail', 'itemprop', 'vehicleInteriorColor'))

['Medium Earth Gray',
 'Pecan',
 'Satellite Grey',
 'Brunell W/black',
 'Graphite',
 'Steel',
 'Titan Black',
 'Graphite',
 'Black',
 'Cocoa/dark Atmosphe',
 'Black Interior',
 'Black/red',
 None,
 'Silk Beige/espresso',
 'Ebony',
 'Black',
 'Beige',
 'Black',
 'Dark Charcoal',
 'Beige',
 'Off-black',
 'Ivory',
 'Black',
 'Black',
 'Ebony',
 'Cafe Latte',
 'Charcoal Black',
 'Charcoal',
 'Black/light Graysto',
 'Jet Black/light Whe',
 'Gray',
 'Beige',
 'Wheat',
 'Redwood',
 'Black',
 'Black',
 'Jet Black',
 'Jet Black',
 'Jet Black',
 'Ash',
 'Black',
 'Jet Black/jet Black',
 'Pale Adobe',
 'Charcoal Black',
 'Black',
 'Gray',
 'Cardamom',
 'Ebony',
 'Charcoal Black',
 'Black Interior']

### Transmission

In [16]:
pi.clean_text_data(
    pi.parse_subsection(soup, 'div', 'li', 'thumbnail', 'list-group-item InvTransmissiontype', 'get_text'), 'Transmission')

['Automatic',
 'Automatic',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Automatic',
 'Manual']

### Engine

In [17]:
pi.clean_text_data(
    pi.parse_subsection(soup, 'div', 'li', 'thumbnail', 'list-group-item InvEnginetype', 'get_text'), 'Engine')

['3.5L V6 Cylinder Engine',
 '6.2L 8 Cylinder Engine',
 '1.5L 3 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '3.0L V6 Cylinder Engine',
 '6.7L 8 Cylinder Engine',
 '1.4L 4 Cylinder Engine',
 '5.7L 8 Cylinder Engine',
 '2.0L 4 Cylinder Engine',
 '6.2L 8 Cylinder Engine',
 '3.6L V6 Cylinder Engine',
 '3.0L V6 Cylinder Engine',
 '5.7L 8 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '5.3L 8 Cylinder Engine',
 '5.6L 8 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '4.0L V6 Cylinder Engine',
 '3.8L V6 Cylinder Engine',
 '2.0L 4 Cylinder Engine',
 '2.4L 4 Cylinder Engine',
 '3.6L V6 Cylinder Engine',
 '3.6L V6 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '2.5L 4 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '3.6L V6 Cylinder Engine',
 '3.6L V6 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '2.4L 4 Cylinder Engine',
 '3.0L V6 Cylinder Engine',
 '4.0L V6 Cylinder Engine',
 '3.5L V6 Cylinder Engine',
 '3.0L V6 Cylinder Engine',
 '5.3L

### Drivetrain

In [18]:
pi.clean_text_data(
    pi.parse_subsection(soup, 'div', 'li', 'thumbnail', 'list-group-item drivetrain-item', 'get_text'), 'Drivetrain')

['Four Wheel Drive',
 'Four Wheel Drive',
 'All Wheel Drive',
 'Four Wheel Drive',
 'All Wheel Drive',
 'Four Wheel Drive',
 'Front Wheel Drive',
 'Four Wheel Drive',
 'All Wheel Drive',
 'Four Wheel Drive',
 'Four Wheel Drive',
 'All Wheel Drive',
 'Rear Wheel Drive',
 'All Wheel Drive',
 'Rear Wheel Drive',
 'Four Wheel Drive',
 'Four Wheel Drive',
 'All Wheel Drive',
 'Four Wheel Drive',
 'All Wheel Drive',
 'All Wheel Drive',
 'Front Wheel Drive',
 'All Wheel Drive',
 'All Wheel Drive',
 'All Wheel Drive',
 'Front Wheel Drive',
 'Front Wheel Drive',
 'Front Wheel Drive',
 'Front Wheel Drive',
 'All Wheel Drive',
 'All Wheel Drive',
 'Front Wheel Drive',
 'All Wheel Drive',
 'Four Wheel Drive',
 'Front Wheel Drive',
 'All Wheel Drive',
 'Four Wheel Drive',
 'Four Wheel Drive',
 'Four Wheel Drive',
 'Front Wheel Drive',
 'All Wheel Drive',
 'Front Wheel Drive',
 'Four Wheel Drive',
 'Front Wheel Drive',
 'All Wheel Drive',
 'All Wheel Drive',
 'All Wheel Drive',
 'Front Wheel Drive',

### VIN

In [19]:
pi.clean_text_data(
    pi.parse_subsection(soup, 'div', 'li', 'thumbnail', 'list-group-item InvVin', 'get_text'), 'VIN')

['1FTEW1EG5GKD26164',
 '1FT8W3B60GEC37377',
 'WMZYV5C35H3B75526',
 '1FMJU1MT2FEF15902',
 'JN1EV7AR1HM840101',
 '1FT7W2BT0FED48828',
 '3VWDB7AJ9JM260615',
 '5TFUY5F10FX423622',
 'TRUK1AFK8D1001374',
 '1GKS2CKJ1FR654928',
 '1C4BJWDG2DL671450',
 'WAUKGAFLXEA021268',
 '1G1YY33P7P5101290',
 'WDDKJ8JB6EF282191',
 '1GCES14P74B107165',
 '1N6AA1E50HN538088',
 '5FPYK16579B101645',
 '2T2BZMCA4GC042676',
 'JTEBU4BF7EK205134',
 'KMHGN4JE1GU141742',
 'YV140MTL1J2462274',
 '1HGCP2F35CA075928',
 '2C3CDZGG9JH289051',
 '2C3CDZGG3KH690340',
 '19UUB3F48LA001789',
 '1N4AA5AP0CC827645',
 '1FA6P0H70E5365477',
 '1N4AA6AP7GC379917',
 '2C4RC1BG4GR110760',
 '2G61V5S36D9130888',
 '5FNYF6H31GB089558',
 '5XXGM4A78FG460844',
 'JN1EV7ARXJM443430',
 'JTEBU5JR4F5241848',
 'JTHBK1GG5E2127042',
 'WAUDGAFL2EA139517',
 '3GTU2UEC0FG341344',
 '1GCVKSEC9GZ238434',
 '1GCGTCEN3K1145395',
 '5TDKZ3DCXJS949090',
 'JM3KFBDM1L0723903',
 '1G1RC6S51JU142077',
 '1FTFW1ET5DKE48455',
 '1FAHP2E85FG187802',
 '2T3RFREV1JW811818',
 '5J6RW2H8

## Check Full Script

### Check Data

In [66]:
c = parse_dealership.get_jm_auto_inventory_data(soup, dealerships['JM Automotive'])

In [67]:
c.head()

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2016 Ford F-150 4WD SuperCrew 145 XLT,2016,Ford,F-150 4WD SuperCrew 145 XLT,None,F-150,4WD SuperCrew 145 XLT,89305,28995,Ingot Silver Metallic,Medium Earth Gray,Automatic,3.5L V6 Cylinder Engine,Four Wheel Drive,1FTEW1EG5GKD26164,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-27 22:19:45.868625
1,2016 Ford Super Duty F-350 Srw 4WD Crew Cab 15...,2016,Ford,Super Duty F-350 Srw 4WD Crew Cab 156 Platinum,None,Super Duty F-350 Srw,4WD Crew Cab 156 Platinum,24752,53995,Shadow Black,Pecan,Automatic,6.2L 8 Cylinder Engine,Four Wheel Drive,1FT8W3B60GEC37377,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-27 22:19:45.868625
2,2017 Mini Countryman Cooper ALL4,2017,Mini,Countryman Cooper ALL4,None,Countryman,Cooper ALL4,77144,18995,Moonwalk Gray Metallic,Satellite Grey,Manual,1.5L 3 Cylinder Engine,All Wheel Drive,WMZYV5C35H3B75526,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-27 22:19:45.868625
3,2012 Toyota Fj Cruiser 4WD 4dr Auto,2012,Toyota,Fj Cruiser 4WD 4dr Auto,None,Fj Cruiser,4WD 4dr Auto,92465,30995,Iceberg,Dark Charcoal,Automatic,4.0L V6 Cylinder Engine,Four Wheel Drive,JTEBU4BF6CK123246,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-27 22:19:45.868625
4,2015 Ford Expedition 4WD 4dr Platinum,2015,Ford,Expedition 4WD 4dr Platinum,None,Expedition,4WD 4dr Platinum,87901,31995,Ruby Red Metallic Tinted Clearcoat,Brunell W/black,Automatic,3.5L V6 Cylinder Engine,Four Wheel Drive,1FMJU1MT2FEF15902,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-27 22:19:45.868625


In [68]:
c.isnull().sum()

title                  0
year                   0
make                   0
model_trim             0
vehicle_type          50
model                  0
trim                   0
vehicle_mileage        0
price                  0
exterior_color         0
interior_color         1
transmission           0
engine                 0
drivetrain             0
vin                    0
dealership_name        0
dealership_address     0
dealership_zipcode     0
dealership_city        0
dealership_state       0
inventory_url          0
scraped_date           0
dtype: int64

In [21]:
## Check Null
c.loc[c.interior_color.isnull(), :]

,title,year,make,model_trim,vehicle_type,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
49,2015 Audi Q7 quattro 4dr 3.0T Premium Plus,2015,Audi,Q7,quattro 4dr 3.0T Premium Plus,84066,26995,Atlantis Blue Metallic,NaN,Automatic,3.0L V6 Cylinder Engine,All Wheel Drive,WA1LGAFE8FD004486,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-24 22:36:11.509159


## Check DB

In [41]:
# Check other null data points
conn = sqlite3.connect('cars.db')
sql_query = "SELECT * FROM test WHERE dealership_name = 'J&M Automotive'"
#sql_query = "SELECT * FROM inventory_staging"
result = pd.read_sql_query(sql_query, conn)

In [42]:
result

,title,year,make,model_trim,vehicle_type,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2016 Ford F-150 4WD SuperCrew 145 XLT,2016,Ford,F-150,4WD SuperCrew 145 XLT,89305,28995,Ingot Silver Metallic,Medium Earth Gray,Automatic,3.5L V6 Cylinder Engine,Four Wheel Drive,1FTEW1EG5GKD26164,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:02.549807
1,2016 Ford Super Duty F-350 Srw 4WD Crew Cab 15...,2016,Ford,Super Duty F-350 Srw,4WD Crew Cab 156 Platinum,24752,53995,Shadow Black,Pecan,Automatic,6.2L 8 Cylinder Engine,Four Wheel Drive,1FT8W3B60GEC37377,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:02.549807
2,2017 Mini Countryman Cooper ALL4,2017,Mini,Countryman,Cooper ALL4,77144,18995,Moonwalk Gray Metallic,Satellite Grey,Manual,1.5L 3 Cylinder Engine,All Wheel Drive,WMZYV5C35H3B75526,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:02.549807
3,2012 Toyota Fj Cruiser 4WD 4dr Auto,2012,Toyota,Fj Cruiser,4WD 4dr Auto,92465,30995,Iceberg,Dark Charcoal,Automatic,4.0L V6 Cylinder Engine,Four Wheel Drive,JTEBU4BF6CK123246,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:02.549807
4,2015 Ford Expedition 4WD 4dr Platinum,2015,Ford,Expedition,4WD 4dr Platinum,87901,31995,Ruby Red Metallic Tinted Clearcoat,Brunell W/black,Automatic,3.5L V6 Cylinder Engine,Four Wheel Drive,1FMJU1MT2FEF15902,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:02.549807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,2014 Toyota Tacoma 4WD Double Cab V6 AT,2014,Toyota,Tacoma,4WD Double Cab V6 AT,53421,28995,Silver Sky Metallic,Graphite,Automatic,4.0L V6 Cylinder Engine,Four Wheel Drive,3TMLU4EN3EM143512,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:24.924182
472,2016 Toyota Tacoma 4WD Access Cab V6 AT TRD Of...,2016,Toyota,Tacoma,4WD Access Cab V6 AT TRD Off Road,36397,36995,Super White,None,None,3.5L V6 Cylinder Engine,Four Wheel Drive,5TFSZ5AN8GX008189,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:24.924182
473,2012 Lincoln Mkz 4dr Sdn AWD,2012,Lincoln,Mkz,4dr Sdn AWD,88925,12995,Steel Blue Metallic,Light Camel,Automatic,3.5L V6 Cylinder Engine,All Wheel Drive,3LNHL2JC9CR810841,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:24.924182
474,2017 Honda Accord LX CVT,2017,Honda,Accord,LX CVT,83396,19995,White Orchid Pearl,Ivory,Automatic,2.4L 4 Cylinder Engine,Front Wheel Drive,1HGCR2F30HA020482,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:29:24.924182


### Before

In [73]:
conn = sqlite3.connect('cars.db')
sql_query = "SELECT dealership_name, count(*) AS num_of_vehicles FROM inventory_staging GROUP BY 1"
result = pd.read_sql_query(sql_query, conn)
result

,dealership_name,num_of_vehicles
0,Avon Auto Brokers,303
1,Blasius Boston,289
2,Bostonyan Auto Group,18
3,Direct Auto Mecca,147
4,Fafama Auto Sales,132
5,Johns Auto Sales,147
6,Newton Automotive Sales,35


### After

In [66]:
conn = sqlite3.connect('cars.db')
sql_query = "SELECT dealership_name, count(*) AS num_of_vehicles FROM inventory_staging GROUP BY 1"
result = pd.read_sql_query(sql_query, conn)
result

,dealership_name,num_of_vehicles
0,Avon Auto Brokers,303
1,Blasius Boston,289
2,Bostonyan Auto Group,18
3,Direct Auto Mecca,147
4,Fafama Auto Sales,132
5,J&M Automotive,476
6,John's Auto Sales,294
7,Newton Automotive Sales,35


### Check Random Null Values to QA

In [67]:
# Check other null data points
conn = sqlite3.connect('cars.db')
sql_query = "SELECT * FROM inventory_staging WHERE dealership_name = 'J&M Automotive'"
#sql_query = "SELECT * FROM inventory_staging"
result = pd.read_sql_query(sql_query, conn)

In [68]:
result.tail()

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
471,2014 Toyota Tacoma 4WD Double Cab V6 AT,2014,Toyota,Tacoma 4WD Double Cab V6 AT,None,Tacoma,4WD Double Cab V6 AT,53421,28995,Silver Sky Metallic,Graphite,Automatic,4.0L V6 Cylinder Engine,Four Wheel Drive,3TMLU4EN3EM143512,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:50:53.953560
472,2016 Toyota Tacoma 4WD Access Cab V6 AT TRD Of...,2016,Toyota,Tacoma 4WD Access Cab V6 AT TRD Off Road,None,Tacoma,4WD Access Cab V6 AT TRD Off Road,36397,36995,Super White,None,None,3.5L V6 Cylinder Engine,Four Wheel Drive,5TFSZ5AN8GX008189,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:50:53.953560
473,2012 Lincoln Mkz 4dr Sdn AWD,2012,Lincoln,Mkz 4dr Sdn AWD,None,Mkz,4dr Sdn AWD,88925,12995,Steel Blue Metallic,Light Camel,Automatic,3.5L V6 Cylinder Engine,All Wheel Drive,3LNHL2JC9CR810841,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:50:53.953560
474,2017 Honda Accord LX CVT,2017,Honda,Accord LX CVT,None,Accord,LX CVT,83396,19995,White Orchid Pearl,Ivory,Automatic,2.4L 4 Cylinder Engine,Front Wheel Drive,1HGCR2F30HA020482,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:50:53.953560
475,2017 Nissan Sentra S CVT,2017,Nissan,Sentra S CVT,None,Sentra,S CVT,82109,13995,Black,None,Automatic,1.8L 4 Cylinder Engine,Front Wheel Drive,3N1AB7AP1HY374009,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:50:53.953560


In [69]:
result.isnull().sum()

title                   0
year                    0
make                    0
model_trim              0
vehicle_type          476
model                   0
trim                    0
vehicle_mileage         0
price                   0
exterior_color          0
interior_color         12
transmission            4
engine                  0
drivetrain              0
vin                     0
dealership_name         0
dealership_address      0
dealership_zipcode      0
dealership_city         0
dealership_state        0
inventory_url           0
scraped_date            0
dtype: int64

In [62]:
result.loc[result.transmission.isnull(), :]

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
263,None,2010,Ford,Super Duty F-250 Srw Super Duty F-250 Srw,None,Super Duty F-250 Srw,Super Duty F-250 Srw,170882,17995,Oxford White,Medium Stone,None,5.4L 8 Cylinder Engine,Four Wheel Drive,1FTSX2B52AEA54798,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:44:08.856191
302,None,2008,Ford,Super Duty F-350 Drw Super Duty F-350 Drw,None,Super Duty F-350 Drw,Super Duty F-350 Drw,13454,37995,Oxford White,Camel,None,6.4L 8 Cylinder Engine,Four Wheel Drive,1FDWF37R58ED29916,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:44:12.383617
382,None,2008,Nissan,Altima Altima,None,Altima,Altima,79330,9995,Dark Slate Metallic,Frost,None,2.5L 4 Cylinder Engine,Front Wheel Drive,1N4AL21E18N417391,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:44:14.763709
472,None,2016,Toyota,Tacoma Tacoma,None,Tacoma,Tacoma,36397,36995,Super White,None,None,3.5L V6 Cylinder Engine,Four Wheel Drive,5TFSZ5AN8GX008189,J&M Automotive,756/820 New Haven Road,06770,Naugatuck,CT,https://www.jmautomotive.com/cars-for-sale-in-...,2022-03-25 22:44:28.720324


In [72]:
TABLE_NAME = 'inventory_staging'
DB_NAME = 'cars.db'

#Connecting to sqlite
conn = sqlite3.connect(DB_NAME)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute(f"DELETE FROM {TABLE_NAME} WHERE dealership_name = 'J&M Automotive'")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()


In [34]:
TABLE_NAME = 'test'
DB_NAME = 'cars.db'

#Connecting to sqlite
conn = sqlite3.connect(DB_NAME)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute(f"DROP TABLE {TABLE_NAME}")
print("Table dropped... ")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()


Table dropped... 


In [22]:
TABLE_NAME = 'parsing_errors_test'
DB_NAME = 'cars.db'

#Connecting to sqlite
conn = sqlite3.connect(DB_NAME)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Doping EMPLOYEE table if already exists
cursor.execute(f"DROP TABLE {TABLE_NAME}")
print("Table dropped... ")

#Commit your changes in the database
conn.commit()

#Closing the connection
conn.close()


Table dropped... 


In [32]:
conn = sqlite3.connect('cars.db')
sql_query = "SELECT * FROM test"
result = pd.read_sql_query(sql_query, conn)
result

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2013 Acura RDX Tech Pkg,2013,Acura,RDX Tech Pkg,SUV,RDX,Tech Pkg,120771,15995.0,Graphite Luster Metallic,Black,Automatic,3.5L V6 Cylinder Engine,All Wheel Drive,5J8TB4H55DL003514,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
1,2007 Aston Martin V8 tage 2dr Conv Sportshift,2007,Aston Martin,V8 tage 2dr Conv Sportshift,Convertible,V8 Vantage,2dr Conv Sportshift,35137,47888.0,Standard Paint,Standard Interior,Automatic,4.3L 8 Cylinder Engine,Rear Wheel Drive,SCFBF04B47GD06725,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
2,2015 Audi A3 1.8T Premium,2015,Audi,A3 1.8T Premium,Sedan,A3,1.8T Premium,64060,18995.0,Brilliant Black,Black,Automatic,1.8L 4 Cylinder Engine,Front Wheel Drive,WAUACGFF3F1141158,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
3,2014 Audi A4 Premium Plus,2014,Audi,A4 Premium Plus,Sedan,A4,Premium Plus,91539,17495.0,Dakota Gray Metallic,Black,8 Speed Automatic,2.0L 4 Cyl.,AWD,WAUFFAFL3EN045649,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
4,2014 Audi A4 Premium,2014,Audi,A4 Premium,Sedan,A4,Premium,80525,18995.0,Brilliant Black,Black,8 Speed Automatic,2.0L 4 Cyl.,AWD,WAUBFAFLXEN003073,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
5,2015 Audi A4 Premium,2015,Audi,A4 Premium,Sedan,A4,Premium,82029,17888.0,Ibis White,Black,Automatic,2.0L 4 Cylinder Engine,All Wheel Drive,WAUBFAFL9FN014552,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
6,2018 Audi A4 Premium,2018,Audi,A4 Premium,Sedan,A4,Premium,58194,28995.0,Glacier White Metallic,Rock Gray/Gray,7 Speed Automatic,2.0L 4 Cyl.,AWD,WAUDNAF49JA140852,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
7,2010 Audi A5 2.0L Premium Plus,2010,Audi,A5 2.0L Premium Plus,Coupe,A5,2.0L Premium Plus,63115,17995.0,Ibis White,Black,Automatic,2.0L 4 Cylinder Engine,All Wheel Drive,WAULFAFR7AA017510,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
8,2013 Audi A5 Premium Plus,2013,Audi,A5 Premium Plus,Convertible,A5,Premium Plus,73192,22995.0,Brilliant Black,Black,Automatic,2.0L 4 Cylinder Engine,All Wheel Drive,WAULFAFH5DN009183,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632
9,2013 Audi A5 Prestige,2013,Audi,A5 Prestige,Coupe,A5,Prestige,110549,NaN,SILVER,,8 Speed Automatic,2.0 4 Cylinder Engine,All Wheel Drive,WAUVFAFR9DA002200,Avon Auto Brokers,159 Memorial Drive,02322,Avon,MA,https://avonautobrokers.com/newandusedcars?cle...,2022-03-29 19:36:39.734632


In [39]:

conn = sqlite3.connect('cars.db')
sql_query = "SELECT * FROM parsing_errors_test"
result = pd.read_sql_query(sql_query, conn)
result

,error,dealership,date,url
0,Data Validation,Avon Auto Brokers,2022-03-29 19:38:44.261129,https://avonautobrokers.com/newandusedcars?cle...
1,Data Validation,Avon Auto Brokers,2022-03-29 19:38:45.234880,https://avonautobrokers.com/newandusedcars?page=2
2,Data Validation,Avon Auto Brokers,2022-03-29 19:38:46.230326,https://avonautobrokers.com/newandusedcars?page=3
3,Data Validation,Avon Auto Brokers,2022-03-29 19:38:47.171345,https://avonautobrokers.com/newandusedcars?page=4
4,Data Validation,Avon Auto Brokers,2022-03-29 19:38:48.128083,https://avonautobrokers.com/newandusedcars?page=5
5,Data Validation,Avon Auto Brokers,2022-03-29 19:38:49.152631,https://avonautobrokers.com/newandusedcars?page=6
6,Data Validation,Avon Auto Brokers,2022-03-29 19:38:50.105888,https://avonautobrokers.com/newandusedcars?page=7
7,Data Validation,Avon Auto Brokers,2022-03-29 19:38:51.068245,https://avonautobrokers.com/newandusedcars?page=8
8,Data Validation,Avon Auto Brokers,2022-03-29 19:38:52.309331,https://avonautobrokers.com/newandusedcars?page=9
9,Data Validation,Avon Auto Brokers,2022-03-29 19:38:53.316220,https://avonautobrokers.com/newandusedcars?pag...


In [77]:
error_df

,error,dealership,date,url
0,Data Validation,JM Automotive,2022-03-27 22:36:23.886094,https://www.jmautomotive.com/inventory.aspx?pg...


In [78]:
def add_data_to_sqlite3(db_name, tbl_name, df):
    # Add data to a SQLite database
    conn = sqlite3.connect(db_name) # 'cars.db'
    df.to_sql(tbl_name, conn, if_exists='append', index=False)   

In [79]:
TABLE_NAME = 'test'
ERROR_TBL_NAME = 'parsing_errors_test'
DB_NAME = 'cars.db'

In [80]:
add_data_to_sqlite3(DB_NAME, ERROR_TBL_NAME, error_df)